# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
# adding pandas also
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )

In [4]:
# 10 countries with the lowest infant mortality rates
# initialize dataframe for capturing rates
df1 = pd.DataFrame(columns=['country', 'infant_mortality'])
row = ''

# infant mortality rates, if they exist, will be in <infant_mortality> tags in <country> tags
for element in document.iterfind('country'):
    if element.find('infant_mortality') is not None:
        row = dict(zip(['country','infant_mortality'], [element.find('name').text, element.find('infant_mortality').text]))
        rows = pd.Series(row)
        df1 = df1.append(rows, ignore_index=True)
        
# infant_mortality column (and several future columns) will need to be converted to numeric
df1[['infant_mortality']] = df1[['infant_mortality']].apply(pd.to_numeric)

df1.head()

,country,infant_mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.90
3,Serbia,6.16
4,Andorra,3.69


In [8]:
# sort to see the lowest (check for a 'tie') at position 10 occurred in separate notebook, no tie (no tie in other questions)
df1.sort_values('infant_mortality')[0:10]

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [7]:
# 10 cities with the largest population
# each city has several different population numbers, corresponding to the year
# df2 captures all years, then we will use most recent value
# we also have to account for the posibility that there are cities of the same name in different countries

df2 = pd.DataFrame(columns=['country', 'city', 'population', 'year'])

for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        i=0
        # some cities may not have a population value
        if len(subelement.findall('population')) > 0:
            for elements in subelement.findall('population'):
                row = dict(zip(['country', 'city', 'population', 'year'], 
                               [element.find('name').text, 
                                subelement.find('name').text, 
                                subelement.findall('population')[i].text, 
                                subelement.findall('population')[i].attrib['year']]))
                rows = pd.Series(row)
                df2 = df2.append(rows, ignore_index=True)
                i += 1

df2[['population']] = df2[['population']].apply(pd.to_numeric)
df2[['year']] = df2[['year']].apply(pd.to_numeric)

# city, country row
df2['city_country'] = df2['city'] + ', ' + df2['country']

df2.head()

,country,city,population,year,city_country
0,Albania,Tirana,192000,1987,"Tirana, Albania"
1,Albania,Tirana,244153,1990,"Tirana, Albania"
2,Albania,Tirana,418495,2011,"Tirana, Albania"
3,Albania,Shkodër,62000,1987,"Shkodër, Albania"
4,Albania,Shkodër,77075,2011,"Shkodër, Albania"


In [10]:
# sort by city_country column, then year (most recent first), then drop duplicates (older population values)

df2 = df2.sort_values(['city_country', 'year'], ascending=[True, False]).drop_duplicates('city_country')
df2.sort_values('population', ascending=False)[0:10]

,country,city,population,year,city_country
3750,China,Shanghai,22315474,2010,"Shanghai, China"
2607,Turkey,Istanbul,13710512,2012,"Istanbul, Turkey"
4303,India,Mumbai,12442373,2011,"Mumbai, India"
1546,Russia,Moskva,11979529,2013,"Moskva, Russia"
3746,China,Beijing,11716620,2010,"Beijing, China"
8208,Brazil,São Paulo,11152344,2010,"São Paulo, Brazil"
3754,China,Tianjin,11090314,2010,"Tianjin, China"
3364,China,Guangzhou,11071424,2010,"Guangzhou, China"
4399,India,Delhi,11034555,2011,"Delhi, India"
3371,China,Shenzhen,10358381,2010,"Shenzhen, China"


In [15]:
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
# df3 will store all data needed to perform the calc and determine the sum

row = ''
df3 = pd.DataFrame(columns=['country', 'population', 'year', 'ethnicgroup', 'ethnicgroup_percent'])

for element in document.iterfind('country'):
    # values for ethnicgroup and population are necessary
    if (len(element.findall('ethnicgroup')) > 0) & (len(element.findall('population')) > 0):
        # for each year / ethnic group, another calculation will be needed, as will another row
        num = len(element.findall('ethnicgroup')) * len(element.findall('population'))
        i = 0
        j = 0
        # iterate through each population for each ethnic groups; counters i and j are for this purpose
        for count in range(num):
            row = dict(zip(['country', 'population', 'year', 'ethnicgroup', 'ethnicgroup_percent'], 
                           [element.find('name').text, 
                            element.findall('population')[i].text, 
                            element.findall('population')[i].attrib['year'], 
                            element.findall('ethnicgroup')[j].text,
                            element.findall('ethnicgroup')[j].attrib['percentage']]))
            rows = pd.Series(row)
            df3 = df3.append(rows, ignore_index=True)
            i += 1
            # once through every population instance, counter i reset to 0 for the next ethnic group
            if i % len(element.findall('population')) == 0:
                i = 0
                j += 1

df3[['population']] = df3[['population']].apply(pd.to_numeric)
df3[['year']] = df3[['year']].apply(pd.to_numeric)
df3[['ethnicgroup_percent']] = df3[['ethnicgroup_percent']].apply(pd.to_numeric)

# column added to make the ethnic population calculation, which is population * percent of given ethnicity
df3['ethnicgroup_total_population'] = df3['population'] * (df3['ethnicgroup_percent'] / 100)

# add column representing ethnic groups per country
df3['ethnicgroup_in_country'] = df3['ethnicgroup'] + ', ' + df3['country']

df3.head(20)

,country,population,year,ethnicgroup,ethnicgroup_percent,ethnicgroup_total_population,ethnicgroup_in_country
0,Albania,1214489,1950,Albanian,95.0,1153764.55,"Albanian, Albania"
1,Albania,1618829,1960,Albanian,95.0,1537887.55,"Albanian, Albania"
2,Albania,2138966,1970,Albanian,95.0,2032017.70,"Albanian, Albania"
3,Albania,2734776,1980,Albanian,95.0,2598037.20,"Albanian, Albania"
4,Albania,3446882,1990,Albanian,95.0,3274537.90,"Albanian, Albania"
5,Albania,3249136,1997,Albanian,95.0,3086679.20,"Albanian, Albania"
6,Albania,3304948,2000,Albanian,95.0,3139700.60,"Albanian, Albania"
7,Albania,3069275,2001,Albanian,95.0,2915811.25,"Albanian, Albania"
8,Albania,2800138,2011,Albanian,95.0,2660131.10,"Albanian, Albania"
9,Albania,1214489,1950,Greek,3.0,36434.67,"Greek, Albania"


In [16]:
# use only the most recent year values per ethnic counts in each country
df3 = df3.sort_values(['year'], ascending=False).drop_duplicates('ethnicgroup_in_country')
df3[['ethnicgroup', 'ethnicgroup_total_population']].groupby(['ethnicgroup']).sum().sort_values('ethnicgroup_total_population', ascending=False)[0:10]

,ethnicgroup_total_population
ethnicgroup,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [18]:
# name and country of a) longest river, b) largest lake and c) airport at highest elevation
# 3 dataframes initialized for each

df4a = pd.DataFrame(columns=['river', 'length'])
df4b = pd.DataFrame(columns=['lake', 'area'])
df4c = pd.DataFrame(columns=['airport', 'elevation'])

# capture river name and length (length may be missing)
row = ''
for element in document.iterfind('river'):
    if element.find('length') is not None:
        row = dict(zip(['river', 'length'], [element.find('name').text, element.find('length').text]))
        rows = pd.Series(row)
        df4a = df4a.append(rows, ignore_index=True)

# capture lake name and area (area may be missing)
row = ''
for element in document.iterfind('lake'):
    if element.find('area') is not None:
        row = dict(zip(['lake', 'area'], [element.find('name').text, element.find('area').text]))
        rows = pd.Series(row)
        df4b = df4b.append(rows, ignore_index=True)

# capture airport and elevation (elevation may be missing)
row = ''
for element in document.iterfind('airport'):
    if element.find('elevation') is not None:
        row = dict(zip(['airport', 'elevation'], [element.find('name').text, element.find('elevation').text]))
        rows = pd.Series(row)
        df4c = df4c.append(rows, ignore_index=True)

df4a[['length']] = df4a[['length']].apply(pd.to_numeric)
df4b[['area']] = df4b[['area']].apply(pd.to_numeric)
df4c[['elevation']] = df4c[['elevation']].apply(pd.to_numeric)

In [19]:
# longest river
df4a.sort_values('length', ascending=False)[0:1]

,river,length
174,Amazonas,6448.0


In [21]:
# largest lake
df4b.sort_values('area', ascending=False)[0:1]

,lake,area
54,Caspian Sea,386400.0


In [22]:
# highest airport
df4c.sort_values('elevation', ascending=False)[0:1]

,airport,elevation
80,El Alto Intl,4063.0
